<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-08 13:17:13
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: 56.03 K (0.4%)
Current PnL: -17.78 L (-11.96%)
CY Booked + Current PnL: -7.92 L (-5.33%)
-------------------
Total profit:  2.90 L
Total loss:  -20.68 L
-------------------
Total Booked + Current PnL: 19.06 L (15.66%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.03%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.49 L (55.96%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DIXON,18940.15,-27.87,73.0,H-MC,8.01,144320.0,29384.0,7202.0,1.04,25.57,4.99,31.83,81.0,4.08,1.09,45.52,X40N,ATH,IT
4,APOLLOHOSP,8285.00,-14.42,57.0,H-LC,10.46,171105.0,14928.0,11173.0,-0.35,9.56,6.53,16.71,22.0,1.34,1.29,28.66,X40N,BTT,HEALTHCARE
79,TTKPRESTIG,770.00,104.48,62.0,M-SC,6.19,88111.0,-12666.0,12758.0,-1.04,-12.57,14.48,0.09,245.0,-0.99,0.66,15.76,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,46.0,M-LC,4.49,213749.0,12270.0,14941.0,-0.35,6.09,6.99,13.51,60.0,0.82,1.61,19.45,XY25,NTT,BANKS
33,HEROMOTOCO,5949.07,3.66,82.0,H-MC,22.06,192518.0,41605.0,15709.0,2.56,27.57,8.16,37.97,93.0,2.65,1.45,59.85,AR,ATH,AUTO


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.80,35.0,H-MC,2.16,134515.0,-1273.0,44027.0,-0.26,-0.94,32.73,31.48,107.0,-0.03,1.01,11.07,X40,ATH,INSURANCE
31,HAVELLS,2069.16,0.34,63.0,H-MC,1.89,248602.0,871.0,74183.0,0.70,0.35,29.84,30.30,92.0,0.01,1.87,13.69,X40,ATH,ELECTRICAL
36,ICICIGI,2260.25,-20.58,41.0,H-MC,3.95,136537.0,404.0,30721.0,0.26,0.30,22.50,22.86,91.0,0.01,1.03,12.95,X40,ATH,INSURANCE
53,PGHH,17973.08,-30.55,54.0,H-MC,4.87,200895.0,75.0,68706.0,-0.75,0.04,34.20,34.25,80.0,0.00,1.51,5.24,X40,ATH,FMCG
88,WIPRO,420.00,-14.25,40.0,M-LC,6.88,150654.0,-291.0,109751.0,-0.28,-0.19,72.85,72.51,53.0,-0.00,1.13,5.82,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-2.80,32.0,H-LC,10.05,222045.0,-19879.0,83156.0,-0.33,-8.22,37.45,26.15,8.0,-0.24,1.67,4.93,X40,ATH,IT
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284444.0,-61512.0,185372.0,-0.73,-17.78,65.17,35.80,1.0,-0.33,2.14,0.77,X40,BTT,IT
85,VBL,671.64,-15.83,39.0,H-LC,6.89,303369.0,-12473.0,125140.0,1.25,-3.95,41.25,35.67,5.0,-0.10,2.28,9.36,X40N,ATH,FMCG
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307700.0,-5188.0,176866.0,-1.26,-1.66,57.48,54.87,3.0,-0.03,2.31,4.88,X40,BTT,IT
45,JIOFIN,387.00,-1.23,43.0,H-LC,13.98,218505.0,1099.0,52397.0,0.89,0.51,23.98,24.61,48.0,0.02,1.64,55.60,XY24,BTT,FINANCE


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-6.65,49.0,H-LC,1.73,231862.0,-14289.0,93487.0,-0.83,-5.81,40.32,32.18,16.0,-0.15,1.74,28.15,X200,ATH,IT
52,NESTLEIND,1377.00,-9.37,57.0,H-LC,2.73,278858.0,13432.0,43362.0,-1.39,5.06,15.55,21.40,11.0,0.31,2.10,12.51,XY25,NTT,FMCG
43,ITC,452.00,-38.34,49.0,H-LC,3.13,199536.0,-602.0,21490.0,0.17,-0.30,10.77,10.44,4.0,-0.03,1.50,5.25,X40,NTT,FMCG
19,CIPLA,1795.00,-20.47,48.0,H-LC,5.74,212172.0,7672.0,33735.0,-0.30,3.75,15.90,20.25,10.0,0.23,1.60,12.81,X40N,BTT,PHARMA
85,VBL,671.64,-15.83,39.0,H-LC,6.89,303369.0,-12473.0,125140.0,1.25,-3.95,41.25,35.67,5.0,-0.10,2.28,9.36,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SHALBY,327.0,1108.93,79.0,M-SC,3.22,164511.0,-16632.0,62103.0,8.17,-9.18,37.75,25.10,235.0,-0.27,1.24,31.84,XY24,NTT,HEALTHCARE
55,RAJESHEXPO,518.0,1769.95,60.0,L-SC,8.25,51023.0,-86154.0,86249.0,6.49,-62.81,169.04,0.07,267.0,-1.00,0.38,26.90,OX40N,NTT,JEWELLERY
84,VALIANTORG,838.0,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS
80,UJJIVANSFB,60.0,89.03,54.0,H-SC,12.95,119610.0,-22869.0,42390.0,4.06,-16.05,35.44,13.70,163.0,-0.54,0.90,41.17,OX40N,NTT,BANKS
75,TATAMOTORS,1065.0,-17.55,65.0,M-LC,2.57,260289.0,-16234.0,126318.0,3.66,-5.87,48.53,39.81,54.0,-0.13,1.96,24.75,XY24,NTT,AUTO


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.32,189360.0,24690.0,43269.0,-2.57,14.99,22.85,41.28,89.0,0.57,1.42,24.17,X40N,ATH,AC
48,LAOPALA,464.00,109.37,44.0,H-SC,3.88,66476.0,-34370.0,60194.0,-2.17,-34.08,90.55,25.61,142.0,-0.57,0.50,23.25,AR,NTT,CERAMICS
12,BATAINDIA,2096.00,-32.15,62.0,M-SC,9.70,101384.0,-27286.0,72581.0,-1.66,-21.21,71.59,35.20,219.0,-0.38,0.76,16.37,X40,NTT,FOOTWEAR
52,NESTLEIND,1377.00,-9.37,57.0,H-LC,2.73,278858.0,13432.0,43362.0,-1.39,5.06,15.55,21.40,11.0,0.31,2.10,12.51,XY25,NTT,FMCG
59,REPCOHOME,880.00,-62.00,34.0,H-SC,8.51,140673.0,-64325.0,209561.0,-1.31,-31.38,148.97,70.85,134.0,-0.31,1.06,15.57,XY24,NTT,FINANCE


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,104.48,62.0,M-SC,6.19,88111.0,-12666.0,12758.0,-1.04,-12.57,14.48,0.09,245.0,-0.99,0.66,15.76,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,89.03,54.0,H-SC,12.95,119610.0,-22869.0,42390.0,4.06,-16.05,35.44,13.70,163.0,-0.54,0.90,41.17,OX40N,NTT,BANKS
46,KANSAINER,340.0,-67.97,57.0,H-SC,3.24,220500.0,-49167.0,85510.0,0.87,-18.23,38.78,13.47,138.0,-0.57,1.66,11.53,XY24,NTT,PAINTS
39,INDIGOPNTS,1408.0,114.84,50.0,M-SC,3.54,140901.0,-33658.0,33689.0,-0.79,-19.28,23.91,0.02,221.0,-1.00,1.06,22.56,OX40N,NTT,PAINTS
18,CERA,9475.0,-20.05,45.0,H-SC,2.55,113976.0,-29827.0,56578.0,-0.14,-20.74,49.64,18.60,149.0,-0.53,0.86,25.78,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-28.69,38.0,H-LC,14.29,284444.0,-61512.0,185372.0,-0.73,-17.78,65.17,35.80,1.0,-0.33,2.14,0.77,X40,BTT,IT
41,INFY,2275.00,-21.03,39.0,H-LC,10.97,307700.0,-5188.0,176866.0,-1.26,-1.66,57.48,54.87,3.0,-0.03,2.31,4.88,X40,BTT,IT
43,ITC,452.00,-38.34,49.0,H-LC,3.13,199536.0,-602.0,21490.0,0.17,-0.30,10.77,10.44,4.0,-0.03,1.50,5.25,X40,NTT,FMCG
85,VBL,671.64,-15.83,39.0,H-LC,6.89,303369.0,-12473.0,125140.0,1.25,-3.95,41.25,35.67,5.0,-0.10,2.28,9.36,X40N,ATH,FMCG
1,ABB,7934.00,-40.74,47.0,H-LC,12.90,247459.0,-14160.0,133380.0,0.74,-5.41,53.90,45.57,7.0,-0.11,1.86,5.30,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,82.0,H-MC,22.06,192518.0,41605.0,15709.0,2.56,27.57,8.16,37.97,93.0,2.65,1.45,59.85,AR,ATH,AUTO
25,DIXON,18940.15,-27.87,73.0,H-MC,8.01,144320.0,29384.0,7202.0,1.04,25.57,4.99,31.83,81.0,4.08,1.09,45.52,X40N,ATH,IT
26,DMART,5201.00,-6.78,64.0,H-LC,16.08,232740.0,48259.0,22110.0,-0.91,26.16,9.50,38.14,38.0,2.18,1.75,39.54,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,44.35,64.0,M-MC,7.53,227872.0,2910.0,164751.0,2.37,1.29,72.30,74.53,192.0,0.02,1.71,32.30,XY24,BTT,STEEL
14,BLUESTARCO,2326.38,9.63,54.0,H-MC,3.32,189360.0,24690.0,43269.0,-2.57,14.99,22.85,41.28,89.0,0.57,1.42,24.17,X40N,ATH,AC
33,HEROMOTOCO,5949.07,3.66,82.0,H-MC,22.06,192518.0,41605.0,15709.0,2.56,27.57,8.16,37.97,93.0,2.65,1.45,59.85,AR,ATH,AUTO
86,VOLTAS,1918.49,1.72,62.0,H-MC,3.68,213510.0,21768.0,74259.0,-0.66,11.35,34.78,50.08,99.0,0.29,1.61,18.92,XY25,ATH,AC
31,HAVELLS,2069.16,0.34,63.0,H-MC,1.89,248602.0,871.0,74183.0,0.70,0.35,29.84,30.30,92.0,0.01,1.87,13.69,X40,ATH,ELECTRICAL


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,3.66,82.0,H-MC,22.06,192518.0,41605.0,15709.0,2.56,27.57,8.16,37.97,93.0,2.65,1.45,59.85,AR,ATH,AUTO
51,NATIONALUM,247.44,-42.83,76.0,H-MC,1.29,113526.0,13022.0,17619.0,0.97,12.96,15.52,30.49,79.0,0.74,0.85,49.56,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-39.40,62.0,M-SC,3.66,102458.0,10960.0,67796.0,0.67,11.98,66.17,86.07,223.0,0.16,0.77,48.79,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.09,49.0,H-SC,12.10,136475.0,13139.0,120630.0,0.78,10.65,88.39,108.45,119.0,0.11,1.03,35.39,AR,ATH,MISC
88,WIPRO,420.00,-14.25,40.0,M-LC,6.88,150654.0,-291.0,109751.0,-0.28,-0.19,72.85,72.51,53.0,-0.00,1.13,5.82,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.37,83340.0,-18870.0,112259.0,0.81,-18.46,134.70,91.37,208.0,-0.17,0.63,35.29,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,38.15,87091.0,-26458.0,66511.0,1.80,-23.30,76.37,35.27,268.0,-0.40,0.65,113.16,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-5.18,43.0,H-SC,15.37,88300.0,-12663.0,107196.0,-0.97,-12.54,121.40,93.63,148.0,-0.12,0.66,31.52,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.80,52.0,H-SC,12.88,93360.0,-4620.0,26225.0,0.14,-4.72,28.09,22.05,152.0,-0.18,0.70,35.71,XR,ATH,FINANCE
84,VALIANTORG,838.00,-327.70,60.0,H-SC,9.03,96876.0,-36410.0,121008.0,4.99,-27.32,124.91,63.47,139.0,-0.30,0.73,62.39,XR,NTT,CHEMICALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-41.65,58.0,L-SC,38.15,87091.0,-26458.0,66511.0,1.80,-23.30,76.37,35.27,268.0,-0.40,0.65,113.16,XR,NTT,HOTELS
7,ATULAUTO,844.00,3932.43,76.0,M-SC,8.64,125623.0,-21745.0,73565.0,-1.20,-14.76,58.56,35.16,236.0,-0.30,0.94,28.31,XY24,NTT,AUTO
33,HEROMOTOCO,5949.07,3.66,82.0,H-MC,22.06,192518.0,41605.0,15709.0,2.56,27.57,8.16,37.97,93.0,2.65,1.45,59.85,AR,ATH,AUTO
62,SAMMAANCAP,326.00,-172.73,67.0,M-SC,5.37,83340.0,-18870.0,112259.0,0.81,-18.46,134.70,91.37,208.0,-0.17,0.63,35.29,XY25,NTT,FINANCE
66,SHALBY,327.00,1108.93,79.0,M-SC,3.22,164511.0,-16632.0,62103.0,8.17,-9.18,37.75,25.10,235.0,-0.27,1.24,31.84,XY24,NTT,HEALTHCARE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,43.60
2,50,73.86


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.16
LC    35.99
MC    25.84
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.10
X40      15.49
X40N     14.20
XY25     10.06
AR        9.52
XR        9.10
OX40N     7.48
MH        1.77
X200      1.74
X5K       1.44
SR        1.09
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    27.66
H-MC    22.80
H-SC    22.65
M-SC    13.92
M-LC     7.24
M-MC     2.72
L-SC     1.59
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.78,-0.64,35.18
IT,13.31,-16.55,74.33
BANKS,7.43,-15.45,64.47
FINANCE,7.42,-22.22,67.28
PAINTS,6.05,-14.77,40.46
MISC,5.88,-17.76,85.26
ELECTRICAL,5.71,-8.49,68.28
AUTO,4.95,-2.67,47.34
AC,3.76,3.17,29.62


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2726505.0,22
AR,1213209.0,10
XR,1081714.0,13
X40,906085.0,10
X40N,545477.0,11
OX40N,464243.0,10
XY25,414862.0,7
SR,197933.0,2
X5K,126319.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2762524.0,27
H-LC,1441080.0,16
M-SC,1393113.0,17
H-MC,1173158.0,17
M-LC,416234.0,5
M-MC,315168.0,2
L-SC,248573.0,3
L-MC,59701.0,1
L-LC,39696.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,926818.0,6
M-SC,XY24,809523.0,7
H-SC,AR,663888.0,5
H-LC,X40,615867.0,5
H-SC,XR,516525.0,6
H-MC,XY24,457376.0,4
H-LC,AR,375264.0,2
M-MC,XY24,315168.0,2
H-LC,X40N,237334.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
